## Importing the Dependencies

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Loading the data

In [4]:
import pandas as pd

url = "../data/benin-malanville.csv"

# If it's a CSV:
df = pd.read_csv(url)
print(df.shape)
df.head()


(525600, 19)


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
0,2021-08-09 00:01,-1.2,-0.2,-1.1,0.0,0.0,26.2,93.4,0.0,0.4,0.1,122.1,0.0,998,0,0.0,26.3,26.2,NaN
1,2021-08-09 00:02,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.6,0.0,0.0,0.0,0.0,0.0,998,0,0.0,26.3,26.2,NaN
2,2021-08-09 00:03,-1.1,-0.2,-1.1,0.0,0.0,26.2,93.7,0.3,1.1,0.5,124.6,1.5,997,0,0.0,26.4,26.2,NaN
3,2021-08-09 00:04,-1.1,-0.1,-1.0,0.0,0.0,26.2,93.3,0.2,0.7,0.4,120.3,1.3,997,0,0.0,26.4,26.3,NaN
4,2021-08-09 00:05,-1.0,-0.1,-1.0,0.0,0.0,26.2,93.3,0.1,0.7,0.3,113.2,1.0,997,0,0.0,26.4,26.3,NaN


# 1. Summary Statistics & Missing-Value Report

In [2]:
print("An overview of the dataset: a random sample of 5 rows")
df.sample(5)

An overview of the dataset: a random sample of 5 rows


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,WSgust,WSstdev,WD,WDstdev,BP,Cleaning,Precipitation,TModA,TModB,Comments
274478,2022-02-15 14:39,831.0,605.1,316.4,853.5,784.9,37.3,6.8,2.8,3.9,0.5,93.7,19.6,993,0,0.0,55.1,48.3,NaN
92451,2021-10-12 04:52,-1.6,-0.4,-1.6,0.0,0.0,24.7,94.5,0.4,1.1,0.5,218.4,4.5,994,0,0.0,23.9,23.3,NaN
431508,2022-06-04 15:49,303.6,0.9,300.6,259.6,252.5,37.0,38.1,3.5,4.6,1.1,288.4,12.6,992,0,0.0,48.0,46.7,NaN
9292,2021-08-15 10:53,912.0,840.0,161.9,851.0,853.0,29.8,69.9,4.0,4.9,0.7,273.1,10.5,996,0,0.0,55.6,49.4,NaN
91856,2021-10-11 18:57,-2.3,-0.6,-2.3,0.0,0.0,32.2,68.7,1.6,1.9,0.4,174.6,5.0,994,0,0.0,31.2,30.6,NaN


In [3]:
print("The description of the numeric columns:")
print(df.describe())

The description of the numeric columns:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB           Tamb             RH             WS  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      228.883576      28.179683      54.487969       2.121113   
std       316.536515       5.924297      28.073069       1.603466   
min         0.000000      11.000000       2.100000       0.000

In [8]:
# Calculate percentage of missing values per column
null_percent = (df.isna().sum() / len(df)) * 100

# Filter columns with >5% missing values
cols_over_5 = null_percent[null_percent > 5]

# Display the result
print("Columns with more than 5% missing values:")
#print(cols_over_5)
# Calculate percent of missing values
null_percent = (df.isna().sum() / len(df)) * 100

# List columns with >5% nulls
cols_over_5 = null_percent[null_percent > 5].index.tolist()

print("Columns with >5% nulls:", cols_over_5)


Columns with more than 5% missing values:
Columns with >5% nulls: ['Comments']


# 2. Outlier Detection & Basic Cleaning

## Check missing values

In [6]:
# Columns to check for missing values
cols_to_check = ["GHI", "DNI", "DHI", "ModA", "ModB", "WS", "WSgust"]

print("Missing values in key columns:")
df[cols_to_check].isna().sum()


Missing values in key columns:


GHI       0
DNI       0
DHI       0
ModA      0
ModB      0
WS        0
WSgust    0
dtype: int64

## Check Incorrect / Impossible Values

In [10]:
print("Incorrect or impossible entries (negative values):")

for col in cols_to_check:
    invalid_rows = df[df[col] < 0]
    count_invalid = len(invalid_rows)
    print(f"{col}: {count_invalid} incorrect values found")


Incorrect or impossible entries (negative values):
GHI: 258847 incorrect values found
DNI: 275987 incorrect values found
DHI: 259182 incorrect values found
ModA: 0 incorrect values found
ModB: 0 incorrect values found
WS: 0 incorrect values found
WSgust: 0 incorrect values found



## Outlier Detection Using IQR Method

In [14]:

print("\nOutlier Detection ")

for col in cols_to_check:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
    print(f"{col}: {len(outliers)} outlier rows")



Outlier Detection 
GHI: 145 outlier rows
DNI: 15196 outlier rows
DHI: 8733 outlier rows
ModA: 98 outlier rows
ModB: 240 outlier rows
WS: 6717 outlier rows
WSgust: 5368 outlier rows


## Z-scores and flag rows with |Z| > 3

In [12]:
from scipy.stats import zscore
z_scores = df[cols_to_check].apply(zscore)

# Flag rows where any column has |Z| > 3
outliers_mask = (np.abs(z_scores) > 3).any(axis=1)
outlier_rows = df[outliers_mask]

print("Number of rows with |Z| > 3:", outliers_mask.sum())
print(outlier_rows)

Number of rows with |Z| > 3: 7740
               Timestamp     GHI    DNI    DHI    ModA    ModB  Tamb    RH  \
670     2021-08-09 11:11   836.0  235.0  610.6   778.8   783.8  30.3  68.2   
671     2021-08-09 11:12  1274.0  698.8  615.2  1210.3  1210.3  30.1  69.6   
672     2021-08-09 11:13   938.0  340.1  612.8   891.1   891.1  30.4  68.4   
673     2021-08-09 11:14   718.5  126.8  593.2   682.6   682.6  30.6  68.2   
674     2021-08-09 11:15  1349.0  771.8  618.0  1281.5  1281.5  30.9  67.1   
...                  ...     ...    ...    ...     ...     ...   ...   ...   
525141  2022-08-08 16:22     4.6   -0.1    4.6     6.1     5.9  21.9  99.0   
525142  2022-08-08 16:23     4.6    0.2    4.6     5.9     5.8  21.9  99.0   
525143  2022-08-08 16:24     4.5   -0.1    4.5     5.8     5.7  21.9  99.0   
525151  2022-08-08 16:32     4.7   -0.1    4.8     6.0     5.9  22.0  99.1   
525155  2022-08-08 16:36     4.8    0.0    4.9     6.0     5.9  22.0  99.1   

         WS  WSgust  WSstdev 

## Handle Missing Values in Key Columns

In [9]:
df_dropped = df.dropna(subset=cols_to_check)
print("Shape after dropping rows with missing values:", df_dropped.shape)

# Option 2: Impute missing values with median
df_imputed = df.copy()
for col in cols_to_check:
    median_value = df_imputed[col].median()
    df_imputed[col].fillna(median_value)

print("Missing values after median imputation:\n", df_imputed[cols_to_check].isna().sum())

Shape after dropping rows with missing values: (525600, 19)
Missing values after median imputation:
 GHI       0
DNI       0
DHI       0
ModA      0
ModB      0
WS        0
WSgust    0
dtype: int64


## saving clean data

In [ ]:
import os

df_clean = df_imputed[~outliers_mask]  # keep only rows where all |Z| <= 3
print("After removing Z-score outliers:", df_clean.shape)
os.makedirs("data", exist_ok=True)
output_file = "../data/benin_clean.csv"
df_clean.to_csv(output_file, index=False)
print("Cleaned dataset saved to:", output_file)

After removing Z-score outliers: (517860, 19)
Cleaned dataset saved to: ../data/benin_clean.csv
